In [2]:
from google.colab import drive # code to access google drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


Step 5.1: Load Data

In [3]:
# STEP 0 — Imports
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
df = pd.read_csv("/content/gdrive/MyDrive/Machine Learning Dataset and project /ecommerce_products_sales.csv")
df.head()


,product_id,title,description,category,price,quantity,order_date,region
0,P0000000,Within finish Republican.,This sports product is made of Cotton and offe...,Sports,246.57,2,2024-02-07,Bahia
1,P0000001,Remember leave family bed doctor agreement.,This electronics product is made of Aluminum a...,Electronics,268.05,3,2023-10-11,São Paulo
2,P0000002,Think article well behavior natural.,This electronics product is made of PU Leather...,Electronics,289.75,2,2021-01-29,São Paulo
3,P0000003,Worker writer person various question election...,This phone accessories product is made of Stee...,Phone Accessories,343.24,5,2023-05-06,Minas Gerais
4,P0000004,Stock learn lawyer quite next.,This books product is made of Cotton and offer...,Books,13.08,5,2024-07-16,Rio Grande do Sul


Step 5.2 — Identify LLM-Eligible Columns

In [4]:
text_columns = ["title", "description"]
df[text_columns].isnull().mean()


,0
title,0.0
description,0.0


Step 5.3 — Define LLM Prompt (CRITICAL)

This is what separates professionals from amateurs.
  Prompt Design (Reusable & Controlled)

In [5]:
SYSTEM_PROMPT = """
You are a data assistant for an e-commerce analytics platform.
Your task is to extract structured attributes from product text.
Return concise, consistent outputs.
"""


In [6]:
USER_PROMPT_TEMPLATE = """
Product Title: {title}
Product Description: {description}

Tasks:
1. Assign a clean normalized category (max 2 words)
2. Generate 3 short product tags
3. Write a 1-line product summary
4. Indicate if product is premium (yes/no)

Return JSON only.
"""


In [17]:
class MockLLM:
    def generate(self, system_prompt, user_prompt):
        # Deterministic, explainable output
        return """
        {
          "normalized_category": "electronics",
          "product_tags": ["portable", "durable", "daily-use"],
          "product_summary": "Reliable electronic product suitable for everyday use.",
          "is_premium_product": "no"
        }
        """


In [21]:
llm = MockLLM()


🔹 Step 5.4 — LLM Call Function (Pseudo-Production)

In [22]:
def extract_genai_features(title, description):
    prompt = USER_PROMPT_TEMPLATE.format(
        title=title,
        description=description
    )

    # PSEUDO-CODE (provider-agnostic)
    response = llm.generate(
        system_prompt=SYSTEM_PROMPT,
        user_prompt=prompt
    )

    return response


Step 5.5 — Apply on Sample

In [23]:
sample_df = df.sample(20, random_state=42)


In [24]:
genai_outputs = sample_df.apply(
    lambda row: extract_genai_features(row["title"], row["description"]),
    axis=1
)


In [27]:
sample_df = df.sample(20, random_state=42)

genai_outputs = sample_df.apply(
    lambda row: extract_genai_features(row["title"], row["description"]),
    axis=1
)

genai_outputs.head()


,0
59770,"\n {\n ""normalized_category"": ..."
21362,"\n {\n ""normalized_category"": ..."
127324,"\n {\n ""normalized_category"": ..."
140509,"\n {\n ""normalized_category"": ..."
144297,"\n {\n ""normalized_category"": ..."


Step 5.6 — Parse Output into Columns

In [28]:
{
  "normalized_category": "electronics",
  "product_tags": ["wireless", "bluetooth", "portable"],
  "product_summary": "Portable wireless speaker with deep bass.",
  "is_premium_product": "yes"
}


{'normalized_category': 'electronics',
 'product_tags': ['wireless', 'bluetooth', 'portable'],
 'product_summary': 'Portable wireless speaker with deep bass.',
 'is_premium_product': 'yes'}

In [29]:
import json

parsed = genai_outputs.apply(json.loads)

genai_df = pd.json_normalize(parsed)
genai_df.head()


,normalized_category,product_tags,product_summary,is_premium_product
0,electronics,"[portable, durable, daily-use]",Reliable electronic product suitable for every...,no
1,electronics,"[portable, durable, daily-use]",Reliable electronic product suitable for every...,no
2,electronics,"[portable, durable, daily-use]",Reliable electronic product suitable for every...,no
3,electronics,"[portable, durable, daily-use]",Reliable electronic product suitable for every...,no
4,electronics,"[portable, durable, daily-use]",Reliable electronic product suitable for every...,no


Step 5.7 — Merge Back to Dataset

In [37]:
df_enriched = sample_df.reset_index(drop=True).join(genai_df)
df_enriched.head()


,product_id,title,description,category,price,quantity,order_date,region,normalized_category,product_tags,product_summary,is_premium_product
0,P0059770,Water through man whatever glass stop actually...,This books product is made of Cotton and offer...,Books,300.54,4,2024-07-16,Rio de Janeiro,electronics,"[portable, durable, daily-use]",Reliable electronic product suitable for every...,no
1,P0021362,Action model other hour little value.,This books product is made of PU Leather and o...,Books,446.46,3,2022-01-07,São Paulo,electronics,"[portable, durable, daily-use]",Reliable electronic product suitable for every...,no
2,P0127324,Through Democrat know enjoy create outside wit...,This books product is made of Aluminum and off...,Books,453.12,1,2022-03-07,São Paulo,electronics,"[portable, durable, daily-use]",Reliable electronic product suitable for every...,no
3,P0140509,Firm body make especially if tree.,This electronics product is made of Aluminum a...,Electronics,74.77,5,2021-09-09,Bahia,electronics,"[portable, durable, daily-use]",Reliable electronic product suitable for every...,no
4,P0144297,Often decide front high Democrat.,This electronics product is made of PU Leather...,Electronics,120.15,1,2021-08-01,Rio de Janeiro,electronics,"[portable, durable, daily-use]",Reliable electronic product suitable for every...,no


In [ ]:
for col in df.columns:
    print(f"'{col}'")


'product_id'
'title'
'description'
'category'
'price'
'quantity'
'order_date'
'region'


data_modeling.ipynb

STEP 6.1 — Normalize product_tags (MANDATORY)

📍

In [42]:
# STEP 6 — DATA MODELING (IMPLEMENTATION)


  # STEP 6.0 — PRE-CHECK (DO NOT SKIP)



df_enriched.info()
df_enriched.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 20 entries, 0 to 19
Data columns (total 12 columns):
 #   Column               Non-Null Count  Dtype         
---  ------               --------------  -----         
 0   product_id           20 non-null     object        
 1   title                20 non-null     object        
 2   description          20 non-null     object        
 3   category             20 non-null     object        
 4   price                20 non-null     float64       
 5   quantity             20 non-null     int64         
 6   order_date           20 non-null     datetime64[ns]
 7   region               20 non-null     object        
 8   normalized_category  20 non-null     object        
 9   product_tags         20 non-null     object        
 10  product_summary      20 non-null     object        
 11  is_premium_product   20 non-null     object        
dtypes: datetime64[ns](1), float64(1), int64(1), object(9)
memory usage: 2.0+ KB


,product_id,title,description,category,price,quantity,order_date,region,normalized_category,product_tags,product_summary,is_premium_product
0,P0059770,Water through man whatever glass stop actually...,This books product is made of Cotton and offer...,Books,300.54,4,2024-07-16,Rio de Janeiro,electronics,"portable, durable, daily-use",Reliable electronic product suitable for every...,no
1,P0021362,Action model other hour little value.,This books product is made of PU Leather and o...,Books,446.46,3,2022-01-07,São Paulo,electronics,"portable, durable, daily-use",Reliable electronic product suitable for every...,no
2,P0127324,Through Democrat know enjoy create outside wit...,This books product is made of Aluminum and off...,Books,453.12,1,2022-03-07,São Paulo,electronics,"portable, durable, daily-use",Reliable electronic product suitable for every...,no
3,P0140509,Firm body make especially if tree.,This electronics product is made of Aluminum a...,Electronics,74.77,5,2021-09-09,Bahia,electronics,"portable, durable, daily-use",Reliable electronic product suitable for every...,no
4,P0144297,Often decide front high Democrat.,This electronics product is made of PU Leather...,Electronics,120.15,1,2021-08-01,Rio de Janeiro,electronics,"portable, durable, daily-use",Reliable electronic product suitable for every...,no


In [43]:
# Convert list → string (safe for hashing & BI tools)
df_enriched["product_tags"] = df_enriched["product_tags"].apply(
    lambda x: ", ".join(x) if isinstance(x, list) else x
)


In [44]:
  # STEP 6.1 — CREATE dim_date



import pandas as pd

df_enriched["order_date"] = pd.to_datetime(df_enriched["order_date"])

dim_date = (
    df_enriched[["order_date"]]
    .drop_duplicates()
    .assign(
        date_id=lambda x: x["order_date"].dt.strftime("%Y%m%d").astype(int),
        year=lambda x: x["order_date"].dt.year,
        month=lambda x: x["order_date"].dt.month,
        day=lambda x: x["order_date"].dt.day
    )
    .rename(columns={"order_date": "date"})
    .reset_index(drop=True)
)



In [45]:


#  STEP 6.2 — CREATE dim_region



dim_region = (
    df_enriched[["region"]]
    .drop_duplicates()
    .reset_index(drop=True)
    .reset_index()
    .rename(columns={"index": "region_id"})
)



In [46]:
  # STEP 6.3 — CREATE dim_product

dim_product = (
    df_enriched[
        [
            "product_id",
            "title",
            "category",
            "normalized_category",
            "product_tags",
            "is_premium_product"
        ]
    ]
    .drop_duplicates()
    .reset_index(drop=True)
)


In [47]:
  # STEP 6.4 — CREATE fact_sales



fact_sales = (
    df_enriched
        .merge(
            dim_date,
            left_on="order_date",
            right_on="date",
            how="left"
        )
        .merge(
            dim_region,
            on="region",
            how="left"
        )
        .assign(
            total_amount=lambda x: x["quantity"] * x["price"]
        )
        .loc[
            :,
            [
                "product_id",
                "date_id",
                "region_id",
                "quantity",
                "price",
                "total_amount"
            ]
        ]
        .rename(columns={"price": "unit_price"})
        .reset_index(drop=True)
)


In [48]:


  # STEP 6.5 — ADD sales_id (SURROGATE KEY)



fact_sales.insert(0, "sales_id", range(1, len(fact_sales) + 1))




  # STEP 6.6 — FINAL VALIDATION (DO NOT SKIP)



dim_product.head()
dim_date.head()
dim_region.head()
fact_sales.head()




assert fact_sales["product_id"].isna().sum() == 0
assert fact_sales["date_id"].isna().sum() == 0
assert fact_sales["region_id"].isna().sum() == 0

In [51]:


#  STEP 6.7 — VALIDATION (VERY IMPORTANT)



assert fact_sales.isnull().sum().sum() == 0
assert dim_product["product_id"].is_unique

In [49]:


#  STEP 6.8 — EXPORT TABLES (FOR DADOSFERA / BONUS)


dim_product.to_csv("dim_product.csv", index=False)
dim_date.to_csv("dim_date.csv", index=False)
dim_region.to_csv("dim_region.csv", index=False)
fact_sales.to_csv("fact_sales.csv", index=False)








In [50]:
dim_product.to_csv("dim_product.csv", index=False)
dim_date.to_csv("dim_date.csv", index=False)
dim_date.to_csv("dim_region.csv", index=False)
fact_sales.to_csv("fact_sales.csv", index=False)
from google.colab import files
files.download("dim_product.csv")
files.download("dim_date.csv")
files.download("dim_region.csv")
files.download("fact_sales.csv")


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

STEP 7 — IMPLEMENTATION (NO ERRORS)

We will:

Run SQL using pandasql

Validate queries

Then document them

 STEP 7.0 — ENABLE SQL ENGINE  



In [62]:
!pip install pandasql

In [63]:


from pandasql import sqldf

pysqldf = lambda q: sqldf(q, globals())

 STEP 7.1 — REGISTER TABLES (REQUIRED)


In [64]:
fact_sales_sql = fact_sales.copy()
dim_product_sql = dim_product.copy()
dim_date_sql = dim_date.copy()
dim_region_sql = dim_region.copy()


 Q1 — Total Revenue by Category (BAR)

In [66]:
# SELECT
#     p.normalized_category,
#     SUM(f.total_amount) AS total_revenue
# FROM fact_sales_sql f
# JOIN dim_product_sql p
#     ON f.product_id = p.product_id
# GROUP BY p.normalized_category
# ORDER BY total_revenue DESC;
q1 = """
SELECT
    p.normalized_category,
    SUM(f.total_amount) AS total_revenue
FROM fact_sales_sql f
JOIN dim_product_sql p
    ON f.product_id = p.product_id
GROUP BY p.normalized_category
ORDER BY total_revenue DESC
"""
pysqldf(q1)



,normalized_category,total_revenue
0,electronics,15710.95


Q2 — Revenue Over Time (LINE)

In [67]:
q2 = """
SELECT
    d.year,
    d.month,
    SUM(f.total_amount) AS monthly_revenue
FROM fact_sales_sql f
JOIN dim_date_sql d
    ON f.date_id = d.date_id
GROUP BY d.year, d.month
ORDER BY d.year, d.month
"""
pysqldf(q2)


,year,month,monthly_revenue
0,2021,5,326.31
1,2021,8,120.15
2,2021,9,373.85
3,2021,12,516.64
4,2022,1,1339.38
5,2022,3,1925.31
6,2022,4,515.08
7,2022,8,201.02
8,2023,2,426.16
9,2023,3,245.04


Q3 — Revenue by Region (MAP / BAR)

In [68]:
q3 = """
SELECT
    r.region,
    SUM(f.total_amount) AS region_revenue
FROM fact_sales_sql f
JOIN dim_region_sql r
    ON f.region_id = r.region_id
GROUP BY r.region
ORDER BY region_revenue DESC
"""
pysqldf(q3)


,region,region_revenue
0,Rio Grande do Sul,4551.89
1,São Paulo,3330.93
2,Bahia,3235.19
3,Paraná,2788.31
4,Rio de Janeiro,1430.78
5,Minas Gerais,373.85


Q4 — Premium vs Non-Premium Sales (PIE)

In [69]:
q4 = """
SELECT
    p.is_premium_product,
    SUM(f.total_amount) AS revenue
FROM fact_sales_sql f
JOIN dim_product_sql p
    ON f.product_id = p.product_id
GROUP BY p.is_premium_product
"""
pysqldf(q4)


,is_premium_product,revenue
0,no,15710.95


Q5 — Top 5 Products by Revenue (TABLE)

In [70]:
q5 = """
SELECT
    p.title,
    SUM(f.total_amount) AS product_revenue
FROM fact_sales_sql f
JOIN dim_product_sql p
    ON f.product_id = p.product_id
GROUP BY p.title
ORDER BY product_revenue DESC
LIMIT 5
"""
pysqldf(q5)


,title,product_revenue
0,Discuss short team exist use American.,2462.00
1,Prove politics chance story sometimes term.,2438.90
2,Magazine run eight.,1731.70
3,Race again door ball have up base.,1472.19
4,Action model other hour little value.,1339.38


STEP 8.2 — PIPELINE IMPLEMENTATION

STAGE 1 — INPUT VALIDATION

In [72]:
 required_columns = [
    "product_id", "price", "quantity", "order_date",
    "region", "normalized_category", "is_premium_product"
]

missing_cols = set(required_columns) - set(df_enriched.columns)
assert not missing_cols, f"Missing columns: {missing_cols}"


STAGE 2 — TRANSFORMATION STEP

In [73]:
df_pipeline = df_enriched.copy()

df_pipeline["order_date"] = pd.to_datetime(df_pipeline["order_date"])
df_pipeline["total_amount"] = df_pipeline["price"] * df_pipeline["quantity"]


STAGE 3 — FEATURE NORMALIZATION

In [74]:
df_pipeline["is_premium_flag"] = df_pipeline["is_premium_product"].map(
    {"yes": 1, "no": 0}
)


STAGE 4 — AGGREGATION

In [75]:
sales_agg = (
    df_pipeline
        .groupby(["normalized_category", "region"])
        .agg(
            total_revenue=("total_amount", "sum"),
            total_units=("quantity", "sum"),
            premium_ratio=("is_premium_flag", "mean")
        )
        .reset_index()
)


STAGE 5 — PIPELINE OUTPUT VALIDATION

In [77]:
sales_agg.head()

,normalized_category,region,total_revenue,total_units,premium_ratio
0,electronics,Bahia,373.85,5,0.0
1,electronics,Minas Gerais,1430.78,6,0.0
2,electronics,Paraná,4551.89,12,0.0
3,electronics,Rio Grande do Sul,2788.31,6,0.0
4,electronics,Rio de Janeiro,3235.19,12,0.0


In [76]:

sales_agg.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6 entries, 0 to 5
Data columns (total 5 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   normalized_category  6 non-null      object 
 1   region               6 non-null      object 
 2   total_revenue        6 non-null      float64
 3   total_units          6 non-null      int64  
 4   premium_ratio        6 non-null      float64
dtypes: float64(2), int64(1), object(2)
memory usage: 372.0+ bytes


In [78]:
assert sales_agg.isna().sum().sum() == 0


STEP 8.3 — PIPELINE FUNCTION

In [79]:
def run_sales_pipeline(df):
    df = df.copy()
    df["order_date"] = pd.to_datetime(df["order_date"])
    df["total_amount"] = df["price"] * df["quantity"]
    df["is_premium_flag"] = df["is_premium_product"].map({"yes": 1, "no": 0})

    return (
        df.groupby(["normalized_category", "region"])
          .agg(
              total_revenue=("total_amount", "sum"),
              total_units=("quantity", "sum"),
              premium_ratio=("is_premium_flag", "mean")
          )
          .reset_index()
    )


In [80]:
pipeline_output = run_sales_pipeline(df_enriched)
pipeline_output.head()


,normalized_category,region,total_revenue,total_units,premium_ratio
0,electronics,Bahia,373.85,5,0.0
1,electronics,Minas Gerais,1430.78,6,0.0
2,electronics,Paraná,4551.89,12,0.0
3,electronics,Rio Grande do Sul,2788.31,6,0.0
4,electronics,Rio de Janeiro,3235.19,12,0.0
